# WEB SCRAPING - EJERCICIO INE

#### IMPORTACION LIBRERIAS

In [1]:
import pandas as pd
import requests

C:\Users\Fabian\AppData\Local\Temp\ipykernel_19712\1058106858.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


#### REQUEST

In [2]:
url = 'http://servicios.ine.es/wstempus/jsCache/es/DATOS_TABLA/33791?tip=AM'
headers = {'User-Agent': 'Mozilla/5.0'}

In [3]:
req = requests.get(url, headers=headers, timeout=800)
raw_data = req.json()

#### OBTENCION DATA TABLES

In [4]:
# obtenemos los datos que usaremos a partir de ahora 
data = []
for i in range(len(raw_data)):
    for j in range(3):
        if raw_data[i]['MetaData'][j]['Variable']['Codigo'] == 'MUN':
            data.append(raw_data[i])
            

In [5]:
# Introducimos los valores únicos de columna 'Origen' en una lista
lista = []
for i in range(len(data)):
    lista.append(data[i]['MetaData'][2]['Variable']['Nombre'])
lista = list(set(lista))

In [6]:
# Creamos otra lista con el valor de origen de cada fila
lista_origen = []
for i in range(len(data)):
    for j in range(3):
        if data[i]['MetaData'][j]['Variable']['Nombre'] in lista:
            lista_origen.append(data[i]['MetaData'][j]['Nombre'])

In [7]:
# creamos los data tables
data_table = []
data_table_years = []
for i in range(len(data)):
        template = {
                'id': raw_data[i]['COD'],
                'Poblacion': raw_data[i]['MetaData'][1]['Nombre'],
                'Origen': lista_origen[i]}
        data_table.append(template)

        dicc_valores = {j['Anyo']: j['Valor'] for j in data[i]['Data']}
        data_table_years.append(dicc_valores)       

#### DATA FRAME

In [13]:
df1, df2 = pd.DataFrame(data_table), pd.DataFrame(data_table_years)
df = pd.concat([df1, df2], axis=1).sort_values(by='Poblacion').reset_index(drop=True)
df


,id,Poblacion,Origen,2022,2021,2020,2019,2018,2017,2016,...,2012,2011,2010,2009,2008,2007,2006,2005,2004,2003
0,PC5352697,Agullana,España,81.0,78.0,69.0,67.0,62.0,67.0,75.0,...,70.0,76.0,76.0,73.0,71.0,67.0,68.0,68.0,67.0,61.0
1,PC5352684,Agullana,Marruecos,1.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,PC5352685,Agullana,Argelia,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,PC5352664,Agullana,África,3.0,3.0,3.0,2.0,2.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,PC5352686,Agullana,Ucrania,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26515,PC5362560,Vilopriu,Bolivia,3.0,4.0,4.0,3.0,3.0,1.0,1.0,...,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
26516,PC5362561,Vilopriu,Argentina,1.0,2.0,1.0,2.0,2.0,2.0,2.0,...,1.0,1.0,0.0,0.0,0.0,0.0,2.0,4.0,2.0,0.0
26517,PC5362543,Vilopriu,América,29.0,33.0,30.0,28.0,25.0,19.0,20.0,...,19.0,14.0,15.0,12.0,13.0,8.0,10.0,13.0,8.0,4.0
26518,PC5362563,Vilopriu,Nigeria,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
